In [58]:
import sys

from common.config import get_config

from common.generate_images_labels import get_images_labels_list
from common.generate_images_labels import write_images_labels_to_file


def main():
    images_source = 'train'
    if images_source not in ['train', 'test']:
        print("Invalid image-source '{}'!".format(images_source))
        return

    images_dir_path = get_config('{}ing_images_dir_path'.format(images_source))
    images_labels_path = get_config(
        '{}ing_images_labels_path'.format(images_source))

    print("Gathering info about images at path '{}'...".format(images_dir_path))
    images_labels_list = get_images_labels_list(images_dir_path)
    print("Done!")

    print("Writing images labels info to file at path '{}'...".format(
        images_labels_path))
    write_images_labels_to_file(images_labels_list, images_labels_path)
    print("Done!")
    
    images_source = 'test'
    if images_source not in ['train', 'test']:
        print("Invalid image-source '{}'!".format(images_source))
        return

    images_dir_path = get_config('{}ing_images_dir_path'.format(images_source))
    images_labels_path = get_config(
        '{}ing_images_labels_path'.format(images_source))

    print("Gathering info about images at path '{}'...".format(images_dir_path))
    images_labels_list = get_images_labels_list(images_dir_path)
    print("Done!")

    print("Writing images labels info to file at path '{}'...".format(
        images_labels_path))
    write_images_labels_to_file(images_labels_list, images_labels_path)
    print("Done!")


if __name__ == '__main__':
    main()

Gathering info about images at path 'C:\Users\Subham\Documents\Indian-Sign-Language\data\images\train'...
Done!
Writing images labels info to file at path 'C:\Users\Subham\Documents\Indian-Sign-Language\data\generated\training_images_labels.txt'...
Done!
Gathering info about images at path 'C:\Users\Subham\Documents\Indian-Sign-Language\data\images\test'...
Done!
Writing images labels info to file at path 'C:\Users\Subham\Documents\Indian-Sign-Language\data\generated\testing_images_labels.txt'...
Done!


In [59]:
"""
Takes a set of images as inputs, transforms them using multiple algorithms to
make it suitable for ingestion into ML routines, then finally outputs them
to disk.
"""
import csv
import traceback

import numpy as np
import cv2

from common.config import get_config
from common.image_transformation import apply_image_transformation


def write_frame_to_file(frame, frame_label, writer):
    """
    Convert the multi-dimensonal array of the image to a one-dimensional one
    and write it to a file, along with its label.
    """
   # print("Writing frame to file...")

    flattened_frame = frame.flatten()
    output_line = [frame_label] + np.array(flattened_frame).tolist()
    writer.writerow(output_line)

    #print("Done!")


def main():
    images_transformed_path = get_config('images_transformed_path')
    with open(images_transformed_path, 'w') as output_file:
        writer = csv.writer(output_file, delimiter=',')

        training_images_labels_path = get_config('training_images_labels_path')
        with open(training_images_labels_path, 'r') as file:
            lines = file.readlines()

        for line in lines:
            #print("\n\n" + line.strip())
            image_path, image_label = line.split()

            # Read the input image.
            frame = cv2.imread(image_path)
            # `frame` is a HxW numpy ndarray of triplets (pixels), where H and W are
            # the dimensions of the input image.
            # cv2.imshow("Original", frame)
            try:
                frame = apply_image_transformation(frame)
                write_frame_to_file(frame, image_label, writer)
            except Exception:
                exception_traceback = traceback.format_exc()
                print("Error while applying image transformation on image path '{}' with the following exception trace:\n{}".format(
                    image_path, exception_traceback))
                continue
            # cv2.waitKey(1000)
    cv2.destroyAllWindows()
    print ("The program completed successfully !!")


if __name__ == '__main__':
    main()

The program completed successfully !!


In [62]:
import os
import sys
import csv
import numpy
from matplotlib import pyplot
from pandas import read_csv
from pandas import set_option
from pandas.tools.plotting import scatter_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
import numpy as np
from sklearn import metrics
from sklearn import linear_model
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.externals import joblib
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import VotingClassifier
import keras
from keras.layers import Dense
from keras.models import Sequential
from common.config import get_config


def print_with_precision(num):
    return "%0.5f" % num



def read_images_transformed(images_transformed_path):
    print("\nReading the transformed images file located at path '{}'...".format(
        images_transformed_path))

    images = []
    labels = []
    with open(images_transformed_path) as images_transformed_file:
        reader = csv.reader(images_transformed_file, delimiter=',')
        cnt = 0
        for line in reader:
            if len(line)==0:
                continue
            cnt = cnt + 1
            label = line[0]
            labels.append(label)
            image = line[1:]
            image_int = [int(pixel) for pixel in image]
            image = np.array(image_int)
            images.append(image)
    print("Done!\n")
    return images, labels


def generate_MLP_classifier():
    print("\nGenerating MLP model ...")
    # Save the number of columns in predictors: n_cols
    n_cols = 900

    # Set up the model: model
#     model = Sequential()

#     # Add the first layer
#     model.add(Dense(32,activation='relu',input_shape=(n_cols,)))

#     # Add the output layer
#     model.add(Dense(1,activation='softmax'))

#     # Compile the model
#     model.compile(optimizer='sgd',loss='categorical_crossentropy',metrics=['accuracy'])
    model = Sequential()
    #model.add(Dense(256, activation="relu", input_dim = X_train.shape[1]))
    model.add(Dense(1, input_dim = n_cols))

    model.compile(loss = "mse", optimizer = "adam")
    print("Done!\n")
    return model

def generate_classifier(model_name):
    classifier_generator_function_name = "generate_{}_classifier".format(
        model_name)
    return globals()[classifier_generator_function_name]()


def divide_data_train_test(images, labels, ratio):
    print("\nDividing dataset in the ratio '{}' using `train_test_split()`:".format(ratio))
    ret = train_test_split(images, labels, test_size=ratio, random_state=0)
    print("Done!\n")
    return ret

In [66]:
model_name ='MLP'
model_output_dir_path = get_config('model_{}_output_dir_path'.format(model_name))
print("Model output directory path: \n"+model_output_dir_path)
model_stats_file_path = os.path.join(model_output_dir_path, "stats-{}.txt".format(model_name))
print("\n\n")
print("Model stats will be written to the file at path \n'{}'.".format(model_stats_file_path))

Model output directory path: 
C:\Users\Subham\Documents\Indian-Sign-Language\data\generated\output\MLP



Model stats will be written to the file at path 
'C:\Users\Subham\Documents\Indian-Sign-Language\data\generated\output\MLP\stats-MLP.txt'.


In [67]:
import pandas as pd
def vec_translate(a):
    my_dict = {'a':0,'b':1,'d':2,'e':3,'f':4,'g':5,'h':6,'j':7,'k':8,'m':9,
               'n':10,'o':11,'p':12,'q':13,'r':14,'s':15,'t':16,'x':17,'y':18,
               'z':19}
    return np.vectorize(my_dict.__getitem__)(a)

with open(model_stats_file_path, "w") as model_stats_file:
    
        images_transformed_path = get_config('images_transformed_path')
        print("Image Transformed Path:: \n"+images_transformed_path)
        
        images, labels = read_images_transformed(images_transformed_path)
        images= StandardScaler().fit_transform(images)
        labels=vec_translate(labels)
#         labels=pd.get_dummies(labels)
#         classifier_model = generate_classifier(model_name)
#         model_stats_file.write("Model used = '{}'".format(model_name))
#         model_stats_file.write("Classifier \n Model details:\n{}\n\n".format(classifier_model))
#         print('IMAGES\n\n')
#         all_images=pd.DataFrame(images)
#         print('\n\n')
#         print('LABELS\n\n')
#         print(labels)
        training_images, testing_images, training_labels, testing_labels = divide_data_train_test(images, labels, 0.2)
        
        print("\nTraining the model...")
        from sklearn.neural_network import MLPClassifier
        clf=MLPClassifier(solver='lbfgs',alpha=1e-5,hidden_layer_sizes=(50,),random_state=1)
        clf.fit(training_images, training_labels)
        #classifier_model = classifier_model.fit(training_images, training_labels)
        print("Done!\n")

        model_serialized_path = get_config('model_{}_serialized_path'.format(model_name))
        print('Model Serialized Path \n')
        print(model_serialized_path)
        print('\n\n')
        print("\nDumping the trained model to disk at path '{}'...".format(model_serialized_path))
#         joblib.dump(classifier_model, model_serialized_path)
        joblib.dump(clf, model_serialized_path)
        print("Dumped\n")
        #print(clf.summary())
        print("\nWriting model stats to file...")
#         score = classifier_model.score(testing_images, testing_labels)
        score = clf.score(testing_images, testing_labels)
        model_stats_file.write("Model score:\n{}\n\n".format(print_with_precision(score)))

#         predicted = classifier_model.predict(testing_images)
        predicted = clf.predict(testing_images)
        print(predicted)
        report = metrics.classification_report(testing_labels, predicted)
        model_stats_file.write("Classification report:\n{}\n\n".format(report))
        print("Done!\n")

        print("\nFinished!\n")

Image Transformed Path:: 
C:\Users\Subham\Documents\Indian-Sign-Language\data\generated\images_transformed.csv

Reading the transformed images file located at path 'C:\Users\Subham\Documents\Indian-Sign-Language\data\generated\images_transformed.csv'...
Done!


Dividing dataset in the ratio '0.2' using `train_test_split()`:
Done!


Training the model...
Done!

Model Serialized Path 

C:\Users\Subham\Documents\Indian-Sign-Language\data\generated\output\MLP\model-serialized-MLP.pkl




Dumping the trained model to disk at path 'C:\Users\Subham\Documents\Indian-Sign-Language\data\generated\output\MLP\model-serialized-MLP.pkl'...
Dumped


Writing model stats to file...
[17  3  3  0 14  2  9 15 10  5  0 14  7 15 19  5  4  1 12 10  5 15  4  9  0
 15  6 19  2 13  7 11 16 10 14 10 16 13 15  0  4 14 14 10  0  3  7 11  5 17
 15  3  5 19 12  2  0  8 12 18 17  9 16  6 18 10  9  7  6 10 16  5  6 16  6
  9 17 13 18 14 15  3  2 18  0  9  6  0  6 16 15  6 12  1 14 15 16 15  0 10
  4  4 10  1  3 11 19 

In [69]:
# model_name = 'MLP_Double_Layer'
model_name='MLP'
model_output_dir_path = get_config('model_{}_output_dir_path'.format(model_name))
print("Model output directory path: \n"+model_output_dir_path)
model_stats_file_path = os.path.join(model_output_dir_path, "stats-{}.txt".format(model_name))
print("\n\n")
print("Model stats will be written to the file at path \n'{}'.".format(model_stats_file_path))
import pandas as pd
def vec_translate(a):
    my_dict = {'a':0,'b':1,'d':2,'e':3,'f':4,'g':5,'h':6,'j':7,'k':8,'m':9,
               'n':10,'o':11,'p':12,'q':13,'r':14,'s':15,'t':16,'x':17,'y':18,
               'z':19}
    return np.vectorize(my_dict.__getitem__)(a)

with open(model_stats_file_path, "w") as model_stats_file:
    
        images_transformed_path = get_config('images_transformed_path')
        print("Image Transformed Path:: \n"+images_transformed_path)
        
        images, labels = read_images_transformed(images_transformed_path)
        images= StandardScaler().fit_transform(images)
        labels=vec_translate(labels)
#         labels=pd.get_dummies(labels)
#         classifier_model = generate_classifier(model_name)
#         model_stats_file.write("Model used = '{}'".format(model_name))
#         model_stats_file.write("Classifier \n Model details:\n{}\n\n".format(classifier_model))
#         print('IMAGES\n\n')
#         all_images=pd.DataFrame(images)
#         print('\n\n')
#         print('LABELS\n\n')
#         print(labels)
        training_images, testing_images, training_labels, testing_labels = divide_data_train_test(images, labels, 0.2)
        
        print("\nTraining the model...")
        from sklearn.neural_network import MLPClassifier
        clf=MLPClassifier(solver='lbfgs',alpha=1e-5,hidden_layer_sizes=(50,50),random_state=1)
        clf.fit(training_images, training_labels)
        #classifier_model = classifier_model.fit(training_images, training_labels)
        print("Done!\n")

        model_serialized_path = get_config('model_{}_serialized_path'.format(model_name))
        print('Model Serialized Path \n')
        print(model_serialized_path)
        print('\n\n')
        print("\nDumping the trained model to disk at path '{}'...".format(model_serialized_path))
#         joblib.dump(classifier_model, model_serialized_path)
        joblib.dump(clf, model_serialized_path)
        print("Dumped\n")
        #print(clf.summary())
        print("\nWriting model stats to file...")
#         score = classifier_model.score(testing_images, testing_labels)
        score = clf.score(testing_images, testing_labels)
        model_stats_file.write("Model score:\n{}\n\n".format(print_with_precision(score)))

#         predicted = classifier_model.predict(testing_images)
        predicted = clf.predict(testing_images)
        print(predicted)
        report = metrics.classification_report(testing_labels, predicted)
        model_stats_file.write("Classification report:\n{}\n\n".format(report))
        print("Done!\n")

        print("\nFinished!\n")

Model output directory path: 
C:\Users\Subham\Documents\Indian-Sign-Language\data\generated\output\MLP



Model stats will be written to the file at path 
'C:\Users\Subham\Documents\Indian-Sign-Language\data\generated\output\MLP\stats-MLP.txt'.
Image Transformed Path:: 
C:\Users\Subham\Documents\Indian-Sign-Language\data\generated\images_transformed.csv

Reading the transformed images file located at path 'C:\Users\Subham\Documents\Indian-Sign-Language\data\generated\images_transformed.csv'...
Done!


Dividing dataset in the ratio '0.2' using `train_test_split()`:
Done!


Training the model...
Done!

Model Serialized Path 

C:\Users\Subham\Documents\Indian-Sign-Language\data\generated\output\MLP\model-serialized-MLP.pkl




Dumping the trained model to disk at path 'C:\Users\Subham\Documents\Indian-Sign-Language\data\generated\output\MLP\model-serialized-MLP.pkl'...
Dumped


Writing model stats to file...
[17  3  3  0 14  2  9 15 10  5  0 14  7 15 19  5  4  1 12 10 11 15  4  9  0
 15  

In [70]:
# model_name = 'MLP_Triple_Layer'
model_name = 'MLP'
model_output_dir_path = get_config('model_{}_output_dir_path'.format(model_name))
print("Model output directory path: \n"+model_output_dir_path)
model_stats_file_path = os.path.join(model_output_dir_path, "stats-{}.txt".format(model_name))
print("\n\n")
print("Model stats will be written to the file at path \n'{}'.".format(model_stats_file_path))
import pandas as pd
def vec_translate(a):
    my_dict = {'a':0,'b':1,'d':2,'e':3,'f':4,'g':5,'h':6,'j':7,'k':8,'m':9,
               'n':10,'o':11,'p':12,'q':13,'r':14,'s':15,'t':16,'x':17,'y':18,
               'z':19}
    return np.vectorize(my_dict.__getitem__)(a)

with open(model_stats_file_path, "w") as model_stats_file:
    
        images_transformed_path = get_config('images_transformed_path')
        print("Image Transformed Path:: \n"+images_transformed_path)
        
        images, labels = read_images_transformed(images_transformed_path)
        images= StandardScaler().fit_transform(images)
        labels=vec_translate(labels)
#         labels=pd.get_dummies(labels)
#         classifier_model = generate_classifier(model_name)
#         model_stats_file.write("Model used = '{}'".format(model_name))
#         model_stats_file.write("Classifier \n Model details:\n{}\n\n".format(classifier_model))
#         print('IMAGES\n\n')
#         all_images=pd.DataFrame(images)
#         print('\n\n')
#         print('LABELS\n\n')
#         print(labels)
        training_images, testing_images, training_labels, testing_labels = divide_data_train_test(images, labels, 0.2)
        
        print("\nTraining the model...")
        from sklearn.neural_network import MLPClassifier
        clf=MLPClassifier(solver='lbfgs',alpha=1e-5,hidden_layer_sizes=(50,50,50),random_state=1)
        clf.fit(training_images, training_labels)
        #classifier_model = classifier_model.fit(training_images, training_labels)
        print("Done!\n")

        model_serialized_path = get_config('model_{}_serialized_path'.format(model_name))
        print('Model Serialized Path \n')
        print(model_serialized_path)
        print('\n\n')
        print("\nDumping the trained model to disk at path '{}'...".format(model_serialized_path))
#         joblib.dump(classifier_model, model_serialized_path)
        joblib.dump(clf, model_serialized_path)
        print("Dumped\n")
        #print(clf.summary())
        print("\nWriting model stats to file...")
#         score = classifier_model.score(testing_images, testing_labels)
        score = clf.score(testing_images, testing_labels)
        model_stats_file.write("Model score:\n{}\n\n".format(print_with_precision(score)))

#         predicted = classifier_model.predict(testing_images)
        predicted = clf.predict(testing_images)
        print(predicted)
        report = metrics.classification_report(testing_labels, predicted)
        model_stats_file.write("Classification report:\n{}\n\n".format(report))
        print("Done!\n")

        print("\nFinished!\n")

Model output directory path: 
C:\Users\Subham\Documents\Indian-Sign-Language\data\generated\output\MLP



Model stats will be written to the file at path 
'C:\Users\Subham\Documents\Indian-Sign-Language\data\generated\output\MLP\stats-MLP.txt'.
Image Transformed Path:: 
C:\Users\Subham\Documents\Indian-Sign-Language\data\generated\images_transformed.csv

Reading the transformed images file located at path 'C:\Users\Subham\Documents\Indian-Sign-Language\data\generated\images_transformed.csv'...
Done!


Dividing dataset in the ratio '0.2' using `train_test_split()`:
Done!


Training the model...
Done!

Model Serialized Path 

C:\Users\Subham\Documents\Indian-Sign-Language\data\generated\output\MLP\model-serialized-MLP.pkl




Dumping the trained model to disk at path 'C:\Users\Subham\Documents\Indian-Sign-Language\data\generated\output\MLP\model-serialized-MLP.pkl'...
Dumped


Writing model stats to file...
[17  3  3  0 14  2  9 15 10  5  0 14  7 15 19  5  4  1 12  9 11 15  4  9 16
 15  

# Testing

In [71]:
"""
Takes a set of images as inputs, transforms them using multiple algorithms to
make it suitable for ingestion into ML routines, then finally outputs them
to disk.
"""
import csv
import traceback

import numpy as np
import cv2

from common.config import get_config
from common.image_transformation import apply_image_transformation


def write_frame_to_file(frame, frame_label, writer):
    """
    Convert the multi-dimensonal array of the image to a one-dimensional one
    and write it to a file, along with its label.
    """
   # print("Writing frame to file...")

    flattened_frame = frame.flatten()
    output_line = [frame_label] + np.array(flattened_frame).tolist()
    writer.writerow(output_line)

    #print("Done!")


def main():
    images_transformed_path = get_config('images_transformed_path')
    with open(images_transformed_path, 'w') as output_file:
        writer = csv.writer(output_file, delimiter=',')

        testing_images_labels_path = get_config('testing_images_labels_path')
        with open(testing_images_labels_path, 'r') as file:
            lines = file.readlines()

        for line in lines:
            #print("\n\n" + line.strip())
            image_path, image_label = line.split()

            # Read the input image.
            frame = cv2.imread(image_path)
            # `frame` is a HxW numpy ndarray of triplets (pixels), where H and W are
            # the dimensions of the input image.
            # cv2.imshow("Original", frame)
            try:
                frame = apply_image_transformation(frame)
                write_frame_to_file(frame, image_label, writer)
            except Exception:
                exception_traceback = traceback.format_exc()
                print("Error while applying image transformation on image path '{}' with the following exception trace:\n{}".format(
                    image_path, exception_traceback))
                continue
            # cv2.waitKey(1000)
    cv2.destroyAllWindows()
    print ("The program completed successfully !!")


if __name__ == '__main__':
    main()

The program completed successfully !!


In [76]:
import pandas as pd
test_input=pd.read_csv('C:\\Users\\Subham\\Documents\\Indian-Sign-Language\\data\\generated\\images_transformed.csv',header=None)
target=test_input[0]
test_input=test_input.drop(0,axis=1)
test_input= StandardScaler().fit_transform(test_input)
target=vec_translate(target)

In [77]:
import sys
import traceback

import cv2
from sklearn.externals import joblib

from common.config import get_config
from common.image_transformation import apply_image_transformation
import warnings

def main():
    warnings.filterwarnings('ignore')
    model_name = 'MLP'
    if model_name not in ['svm', 'logistic', 'knn','DecisionTree','RandomForest','GaussianNB','LinearDiscriminantAnalysis', 
                      'Bagging' , 'ExtraTrees', 'AdaBoost' , 'Voting','GradientBoosting','MLP']:
        print("Invalid model-name '{}'!".format(model_name))
        return

    #print("Using model '{}'...".format(model_name))

    model_serialized_path = get_config(
        'model_{}_serialized_path'.format(model_name))
    #print("Model deserialized from path '{}'".format(model_serialized_path))

    testing_images_labels_path = get_config('testing_images_labels_path')
    total=len(target)
    cnt=0
    classifier_model = joblib.load(model_serialized_path)
    predicted_labels = classifier_model.predict(test_input)
#     predicted_label = predicted_labels[0]
    #print(predicted_labels)
    for i in range(total):
        if target[i] != predicted_labels[i]:
                    cnt += 1
    print(str(cnt)+" "+str(total))
    print("Accuracy of "+model_name+" Classification on Test Data is: "+str((total-cnt)/total))
    cv2.destroyAllWindows()
    #print ("The program completed successfully !!")


if __name__ == '__main__':
    main()

369 450
Accuracy of MLP Classification on Test Data is: 0.18
